Link: [geeksforgeeks](https://www.geeksforgeeks.org/machine-learning/multi-armed-bandit-problem-in-reinforcement-learning/)

[github](https://gibberblot.github.io/rl-notes/single-agent/multi-armed-bandits.html)

**Bài toán**: 
- Có $N$ máy đánh bạc một tay (loại máy có "cánh tay" bên cạnh để người chơi kéo để máy chạy lại)
- Sau mỗi lần kéo máy sẽ trả lại một phần thưởng từ một phân phối xác suất chưa xác định 
- Một số máy có phần thưởng cao hơn các máy khác (nhưng ta không biết)
- Số lần kéo thường là hữu hạn

**Mục tiêu**: Tối đa hóa phần thưởng nhận được 

### Epsilon-Greedy: 
- Mô phỏng bài toán Multi-Armed Bandit 
- Triển khai thuật toán Epsilon-Greedy: 
<br> Với xác suất $\epsilon$ ($\epsilon$ nhỏ), chọn một cánh tay ngẫu nhiên 
<br> Ngược lại, chọn hành động tham lam 



In [1]:
import numpy as np

In [2]:
class EpsilonGreedy:
    ''' 
    Đại diện cho một policy: Chọn một hành động khám phá với xác suất nhỏ, 
    còn lại là tham lam 
    '''
    def __init__(self, n_arms, epsilon):
        ''' 
        Hàm dựng: 
        - n_arms: số cánh tay có thể chọn
        - epsilon: xác suất chọn hành động khám phá
        - counts([]): mảng đếm, để tính values 
        - values([])
        '''
        self.n_arms = n_arms
        self.epsilon = epsilon
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
    
    def select_arm(self):
        '''
        Xác suất explore: epsion, nên xs chọn một hành động trong TH này là epsilon/n => lấy pp đều 
        Còn lại: chọn hành động tham lam 
        '''
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, self.n_arms)
        else:
            return np.argmax(self.values)
    
    def update(self, chosen_arm, reward):
        ''' 
        Với arm được chọn: V[t] = (n-1)/n * V[t-1] + 1/n * R
        Các arm khác không thay đổi 
        '''
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        value_past = self.values[chosen_arm]
        self.values[chosen_arm] = (n-1) / n * value_past + 1/n * reward

In [11]:
''' Mẫu chạy thử '''
n_arms = 10
epsilon = 0.1 
T = 1000

''' 
Thiết lập trạng thái ban đầu 
true_means : kỳ vọng reward của một máy 
'''
true_means = np.random.randn(n_arms)
print(f"Actual mean rewards for each arm: \n {np.round(true_means, 2)}")
print(f"The best arm is #{np.argmax(true_means)} with a mean of {np.max(true_means):.2f} \n")

Actual mean rewards for each arm: 
 [ 1.    0.25 -1.84  2.44  1.17 -0.36 -0.44 -0.47  1.01  2.21]
The best arm is #3 with a mean of 2.44 



In [17]:
# Test voi T = 1000
''' reward: giả định tuân theo phân phối chuẩn, kỳ vọng = true_mean '''
agent = EpsilonGreedy(n_arms, epsilon)
total_reward = 0

for t in range(T):
    arm_to_pull = agent.select_arm()
    reward = np.random.randn() + true_means[arm_to_pull]
    agent.update(arm_to_pull, reward)
    total_reward += reward

print(f"Values: {np.round(agent.values, 2)} \n")
print(f"Total Reward: {total_reward:.2f}")

Var = sum((agent.values-true_means)**2) / (n_arms -1)
print(f"Var: ", round(Var,4))

Values: [ 0.81 -0.04 -2.35  2.12  1.18 -0.13 -0.64 -0.16  0.63  2.24] 

Total Reward: 2080.33
Var:  0.0906


In [18]:
# Test voi T = 10000
agent = EpsilonGreedy(n_arms, epsilon)
total_reward = 0

for t in range(10000):
    arm_to_pull = agent.select_arm()
    reward = np.random.randn() + true_means[arm_to_pull]
    agent.update(arm_to_pull, reward)
    total_reward += reward

print(f"Values: {np.round(agent.values, 2)} \n")
print(f"Total Reward: {total_reward:.2f}")

Var = sum((agent.values-true_means)**2) / (n_arms -1)
print(f"Var: ", round(Var,4))

Values: [ 1.04  0.06 -1.87  2.44  1.27 -0.44 -0.55 -0.52  1.08  2.18] 

Total Reward: 22165.04
Var:  0.0085


In [19]:
# Test voi T = 100000 
agent = EpsilonGreedy(n_arms, epsilon)
total_reward = 0

for t in range(100000):
    arm_to_pull = agent.select_arm()
    reward = np.random.randn() + true_means[arm_to_pull]
    agent.update(arm_to_pull, reward)
    total_reward += reward

print(f"Values: {np.round(agent.values, 2)} \n")
print(f"Total Reward: {total_reward:.2f}")

Var = sum((agent.values-true_means)**2) / (n_arms -1)
print(f"Var: ", round(Var,4))

Values: [ 1.01  0.24 -1.84  2.44  1.11 -0.37 -0.46 -0.48  1.04  2.18] 

Total Reward: 224287.53
Var:  0.0006
